In [ ]:
# https://github.com/lakshmanok/lakblogs/tree/main/bridge_bidding_advisor
# in order to execute the scripts, the supporting indexes 
# are required which in turn requires installation of 
# torch and transformers. Chromadb install will lead to pulling 
# all the supporting libraries. 

In [ ]:
!pip install -r requirements.txt

In [ ]:
# building the vector index
import chromadb
import bs4
import requests
import shutil
import os
import re
from langchain.text_splitter import SentenceTransformersTokenTextSplitter

In [ ]:
URL = "https://www.bridgeworld.com/pages/readingroom/bws/bwscompletesystem.html"
URL_LOCAL = URL.split('/')[-1]
CHROMA_COLLECTION_NAME = "bridge_world_system"
CHROMADB_DIR = "db/"

In [ ]:
def download_file(url: str) -> str:
    local_filename = URL_LOCAL
    if not os.path.exists(local_filename):
        print(f"Downloading {URL} to {local_filename}.")
        with requests.get(url, stream=True) as r:
            with open(local_filename, 'wb') as f:
                shutil.copyfileobj(r.raw, f)
    else:
        print(f"Using already downloaded {local_filename}.")
    return local_filename

In [ ]:
chroma_client = chromadb.PersistentClient(path=CHROMADB_DIR)
collection = chroma_client.get_or_create_collection(name=CHROMA_COLLECTION_NAME)
text_splitter = SentenceTransformersTokenTextSplitter()


In [ ]:
# update paragraphs into chromadb collection
download_file(URL)

# initiate building the index
with open(URL_LOCAL, 'r') as f:
    soup = bs4.BeautifulSoup(f.read(), 'html.parser')
    last_header = ""
    paragraphs = soup.find_all("p")
    for n, paragraph in enumerate(paragraphs):
        paragraph_id = f"{URL_LOCAL}_{n}"
        text = paragraph.text.strip()
        # find the previous header
        header = paragraph.find_all(re.compile("^h[1-5]$"))
        if header:
            header = header[0].text.strip()
            last_header = header
        else:
            header = last_header
        # print(paragraph_id, "->", header, "->", len(text), "->", text[:30])
        # split the text into chunks and insert into chromadb
        ids = []
        documents = []
        metadatas = []
        chunks = text_splitter.create_documents([text]) # takes array of documents
        for chunk_no, chunk in enumerate(chunks):
            ids.append(f"{paragraph_id}#{chunk_no}")
            documents.append(chunk.page_content)
            metadatas.append({"title": header, "source": URL})
        if ids:
            collection.upsert(ids=ids, documents=documents, metadatas=metadatas)
        
        print(f"{int(0.5 + 100.0 * n / len(paragraphs))}% ({collection.count()})", end=" ", flush=True)
        
        if n % 10 == 0:
            print()

In [ ]:
import os
import dotenv
import dspy

def init_gemini_pro(temperature: float = 0.0):
    """
    Initializes dspy to use Gemini as the language model.
    """
    dotenv.load_dotenv("D:\\gitFolders\\python_de_learners_data\\.env")
    api_key = os.getenv("GOOGLE_API_KEY")
    gemini = dspy.Google("models/gemini-1.0-pro",
                         api_key=api_key,
                         temperature=temperature)
    dspy.settings.configure(lm=gemini, max_tokens=1024)


def init_gpt35(temperature: float = 0.0):
    """
    Initializes dspy to use OpenAI GPT 3.5 as the language model.
    """
    dotenv.load_dotenv("D:\\gitFolders\\python_de_learners_data\\.env")
    api_key = os.getenv("OPENAI_API_KEY")
    gpt35 = dspy.OpenAI(model="gpt-3.5-turbo",
                        api_key=api_key,
                        temperature=temperature)
    dspy.settings.configure(lm=gpt35, max_tokens=1024)

In [ ]:
from dspy import teleprompt
from dspy.retrieve.chromadb_rm import ChromadbRM
import json

In [ ]:
class ZeroShot(dspy.Module):
    """
    Provide answer to question
    """
    def __init__(self):
        super().__init__()
        self.prog = dspy.Predict("question -> answer")

    def forward(self, question):
        return self.prog(question="In the game of bridge, " + question)


class Definitions(dspy.Module):
    """
    Retrieve the definition from Wikipedia (2017 version)
    """
    def __init__(self):
        super().__init__()
        self.retriever = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')

    def forward(self, term):
        result = self.retriever(f"In the game of bridge, what does {term} mean?", k=1)
        if result:
            return result[0].long_text
        return ""


class FindTerms(dspy.Module):
    """
    Extract bridge terms from a question
    """
    def __init__(self):
        super().__init__()
        self.entity_extractor = dspy.Predict("question -> terms")

    def forward(self, question):
        max_num_terms = max(1, len(question.split())//4)
        prompt = f"Identify up to {max_num_terms} terms in the following question that are jargon in the card game bridge."
        prediction = self.entity_extractor(
            question=f"{prompt}\n{question}"
        )
        answer = prediction.terms
        if "Terms: " in answer:
            start = answer.rindex("Terms: ") + len("Terms: ")
            answer = answer[start:]
        return [a.strip() for a in answer.split(',')]


def BiddingSystem():
    """
    Retreives rules for bidding in bridge.
    This is just a retriever and does not have any language model.
    """
    from chromadb.utils import embedding_functions
    default_ef = embedding_functions.DefaultEmbeddingFunction()
    return ChromadbRM(CHROMA_COLLECTION_NAME, CHROMADB_DIR, default_ef, k=3)


class AdvisorSignature(dspy.Signature):
    definitions = dspy.InputField(format=str)  # function to call on input to make it a string
    bidding_system = dspy.InputField(format=str) # function to call on input to make it a string
    question = dspy.InputField()
    answer = dspy.OutputField()

def shorten_list(response):
    if type(response) == list:
        return [ f"{r['long_text'][:25]} ... {len(r['long_text'])}" for r in response]
    else:
        return response   
    

class BridgeBiddingAdvisor(dspy.Module):
    """
    Functions as the orchestrator. All questions are sent to this module.
    """
    def __init__(self):
        super().__init__()
        self.find_terms = FindTerms()
        self.definitions = Definitions()
        # self.bidding_system = BiddingSystem()
        self.prog = dspy.ChainOfThought(AdvisorSignature,
                                        n=3)

    def forward(self, question):
        print("a:", question)
        terms = self.find_terms(question)
        print("b:", terms)
        definitions = [self.definitions(term) for term in terms]
        print("c:", definitions)
        bidding_system = BiddingSystem()(question)
        print("d:", shorten_list(bidding_system))
        prediction = self.prog(definitions=definitions,
                               bidding_system=bidding_system,
                               question="In the game of bridge, " + question,
                               max_tokens=-1024)
        return prediction.answer

In [ ]:
def run(name: str, module: dspy.Module, queries: [str], shorten: bool = False):
    print(f"**{name}**")
    for query in queries:
        response = module(query)
        if shorten:
            response = shorten_list(response)
        print(response)
    print()

In [ ]:
questions = [
        "What is Stayman?",
        "When do you use Jacoby Transfers?",
        "Playing Stayman and Transfers, what do you bid with 5-4 in the majors?"
    ]

In [ ]:
run("Zeroshot", ZeroShot(), questions)
# exit(0)

In [ ]:
run("definitions", Definitions(), ["Stayman", "Jacoby Transfers", "Strong 1NT", "majors"])

In [ ]:
run("find_terms", FindTerms(), questions)

In [ ]:
run("bidding_system", BiddingSystem(), questions, shorten=True)

In [ ]:
run("bidding_advisor", BridgeBiddingAdvisor(), questions)

In [ ]:
# create labeled training dataset
traindata = json.load(open("trainingdata.json", "r"))['examples']
trainset = [dspy.Example(question=e['question'], answer=e['answer']) for e in traindata]

In [ ]:
# train
teleprompter = teleprompt.LabeledFewShot()
optimized_advisor = teleprompter.compile(student=BridgeBiddingAdvisor(),
                                         trainset=trainset)
run("optimized", optimized_advisor, questions)